<a href="https://colab.research.google.com/github/mtwenzel/image-video-understanding/blob/master/Section_4_DCGAN_for_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deep Convolutional Generative Adversarial Network

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/dcgan">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/generative/dcgan.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate images of handwritten digits using a [Deep Convolutional Generative Adversarial Network](https://arxiv.org/pdf/1511.06434.pdf) (DCGAN). The code is written using the [Keras Sequential API](https://www.tensorflow.org/guide/keras) with a `tf.GradientTape` training loop.

## What are GANs?
[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661) (GANs) are one of the most interesting ideas in computer science today. Two models are trained simultaneously by an adversarial process. A *generator* ("the artist") learns to create images that look real, while a *discriminator* ("the art critic") learns to tell real images apart from fakes.

![A diagram of a generator and discriminator](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gan1.png?raw=1)

During training, the *generator* progressively becomes better at creating images that look real, while the *discriminator* becomes better at telling them apart. The process reaches equilibrium when the *discriminator* can no longer distinguish real images from fakes.

![A second diagram of a generator and discriminator](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gan2.png?raw=1)

This notebook demonstrates this process on the MNIST dataset. The following animation shows a series of images produced by the *generator* as it was trained for 50 epochs. The images begin as random noise, and increasingly resemble hand written digits over time.

![sample output](https://tensorflow.org/images/gan/dcgan.gif)

To learn more about GANs, we recommend MIT's [Intro to Deep Learning](http://introtodeeplearning.com/) course.

### Import TensorFlow and other libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
import tensorflow as tf

In [0]:
tf.__version__

In [0]:
# To generate GIFs
!pip install imageio

In [0]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

### Load and prepare the dataset

You will use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.

In [0]:
#(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
(train_images, train_labels), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]
BUFFER_SIZE = 60000
BATCH_SIZE = 256
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Create the models

Both the generator and discriminator are defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).

### The Generator

The generator uses `tf.keras.layers.Conv2DTranspose` (upsampling) layers to produce an image from a seed (random noise). Start with a `Dense` layer that takes this seed as input, then upsample several times until you reach the desired image size of 28x28x1. Notice the `tf.keras.layers.LeakyReLU` activation for each layer, except the output layer which uses tanh.

In [0]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

Use the (as yet untrained) generator to create an image.

In [0]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

### The Discriminator

The discriminator is a CNN-based image classifier.

In [0]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

Use the (as yet untrained) discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.

In [0]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [0]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [0]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, we will compare the discriminators decisions on the generated images to an array of 1s.

In [0]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop



In [0]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [0]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

**Generate and save images**



In [0]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## Train the model
Call the `train()` method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real. After about 50 epochs, they resemble MNIST digits. This may take about one minute / epoch with the default settings on Colab.

In [0]:
train(train_dataset, EPOCHS)

Restore the latest checkpoint.

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create a GIF


In [0]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [0]:
display_image(EPOCHS)

Use `imageio` to create an animated gif using the images saved during training.

In [0]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)

If you're working in Colab you can download the animation with the code below:

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(anim_file)

## Next steps


This tutorial has shown the complete code necessary to write and train a GAN. As a next step, you might like to experiment with a different dataset, for example the Large-scale Celeb Faces Attributes (CelebA) dataset [available on Kaggle](https://www.kaggle.com/jessicali9530/celeba-dataset). To learn more about GANs we recommend the [NIPS 2016 Tutorial: Generative Adversarial Networks](https://arxiv.org/abs/1701.00160).


# GAN for Segmentation
Copyright by Fraunhofer MEVIS (2020)

## Data Preparation (the well-known CTs)

In [0]:
%%bash 
test -e tmp_slices.npz || curl -L "https://drive.google.com/uc?export=download&id=1R2-H0dhhrj6XNK7Q-MazIWGeFDOf6Zya" --output tmp_slices.npz

In [0]:
TRAINING_SLICE_COUNT = 600 #@param {min:100, max:3300, step:100}
EPOCHS = 50
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

loaded = np.load('tmp_slices.npz')

x_train = loaded['x_train'][:TRAINING_SLICE_COUNT]
y_train = loaded['y_train'][:TRAINING_SLICE_COUNT]

x_test = loaded['x_train'][TRAINING_SLICE_COUNT:]
y_test = loaded['y_train'][TRAINING_SLICE_COUNT:]

assert len(x_train) == len(y_train)
print("Found %d training and %d testing slices with shape %s" % (len(x_train),len(x_test), x_train.shape))

In [0]:
# remove the lesion labels (values 2..3)
y_train_binary = y_train.clip(0, 1)
y_test_binary = y_test.clip(0, 1)

f, ax = plt.subplots(1, 2, figsize = (9, 5))
imgplot = ax[0].imshow(x_test[example_test_slice])
ax[0].set_title('orig')
imgplot = ax[1].imshow(y_test_binary[example_test_slice])
ax[1].set_title('mask')
plt.show()

## Some auxiliary functions
This is also copy-and-paste from the previous Notebooks.

In [0]:
from tensorflow.keras.callbacks import Callback

def pad_image_for_model(model, input_image):
    '''Determine the necessary amount of padding
    (difference between input and output size of the model)
    and apply it to an ndarry with one or more images.'''
    
    padding = 0
    if model.get_layer('firstConvolutionalLayer').padding == 'valid':
        padding = 20 # WARNING: Hard-coded for above architecture!

        # determine in which dimension to apply this padding
        ndim_padding = []
        if np.ndim(input_image) > 2:
            # do not pad along batch dimension (if present)
            ndim_padding.append((0, 0))
        ndim_padding.append((padding, padding)) # pad above/below image (y dimension)
        ndim_padding.append((padding, padding)) # pad left/right of image (x dimension)
        if np.ndim(input_image) > 3:
            # do not pad along channel dimension (if present)
            ndim_padding.append((0, 0))
        
        input_image = np.lib.pad(input_image, ndim_padding,
                                 #'constant', constant_values = 0)
                                 'reflect')

    return input_image, padding

class VisualHistory(Callback):
    def on_train_begin(self, logs={}):
        # also show initial prediction
        plot_prediction(self.model, example_test_slice)
    
    def on_epoch_end(self, batch, logs={}):
        # show prediction after every training epoch
        plot_prediction(self.model, example_test_slice)
        
def do_prediction(model, input_image, verbose = False):
    # first do padding of full slice
    input_image, padding = pad_image_for_model(model, input_image)
    
    # add batch and channel dimensions (network expects 4D arrays)
    input_array = input_image[np.newaxis,:,:,np.newaxis]
    if verbose:
        print("input shape:", input_array.shape)

    y_predicted = model.predict(input_array)
    if verbose:
        print("output shape:", y_predicted.shape)

    return input_image, input_array, y_predicted, padding

def plot_prediction(model, pred_slice_index):
    # get single slice
    input_image    = x_test[pred_slice_index]
    # could use y_train_binary here for the first half of the notebook, but in the end we want to see the lesion
    reference_mask = y_test[pred_slice_index]

    input_image, input_array, y_predicted, padding = do_prediction(model, input_image)
    
    padded_extent = np.array([0, input_array.shape[2], input_array.shape[1], 0]) - 0.5 - padding

    # display prediction for inspection
    f, ax = plt.subplots(1, 5 if padding else 4, figsize = (11 if padding else 8, 3), sharey = True)
    ax[0].imshow(x_test[pred_slice_index])
    ax[0].set_title('orig')
    if padding:
        ax[1].imshow(input_array[0,:,:,0], extent = padded_extent)
        ax[1].set_title('padded input')
    ax[-2].imshow(y_predicted[0,:,:,0])
    ax[-2].set_title('predicted mask')
    ax[-3].imshow(reference_mask.clip(0,1))
    ax[-3].set_title('reference mask')
    ax[-1].imshow(reference_mask.clip(0,1) - y_predicted[0,:,:,0])
    ax[-1].set_title('(ref - predicted)')
    ax[0].set_ylim(*padded_extent[2:])
    plt.show()

## Modify Data for GAN training from above.

In [0]:
BUFFER_SIZE = TRAINING_SLICE_COUNT 
BATCH_SIZE = 16

# Batch and shuffle the data
train_dataset_liver = tf.data.Dataset.from_tensor_slices( # A tuple of X and Y to shuffle and batch together.
    (np.lib.pad(x_train[...,np.newaxis], [(0,0), (20,20), (20,20), (0,0)], 'reflect'), 
    y_train_binary[...,np.newaxis]))\
    .shuffle(BUFFER_SIZE)\
    .batch(BATCH_SIZE)    

## Define new generator and discriminator

In [0]:
def make_generator_model_seg(_filters=32, filters_add=0, _kernel_size=(3,3), _padding='same', _activation='relu', _kernel_regularizer=None, _final_layer_nonlinearity='sigmoid'):
    model = tf.keras.Sequential()
    # We are indifferent about the xy size, but accept only one channel (gray value images). This has the consequence that debugging sizes gets harder.
    model.add(layers.InputLayer(input_shape=(116,116,1))) 
    
    model.add(layers.Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, name='firstConvolutionalLayer'))
    model.add(layers.Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.MaxPool2D())

    model.add(layers.Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.MaxPool2D())

    model.add(layers.Conv2D(filters=_filters+2*filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.Conv2D(filters=_filters+2*filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.UpSampling2D())

    model.add(layers.Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.UpSampling2D())

    model.add(layers.Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(layers.Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))

    model.add(layers.Conv2D(1, kernel_size=(1,1), activation=_final_layer_nonlinearity))
    return model

In [0]:
def make_discriminator_model_seg():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='valid',
                                     input_shape=[76, 76, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='valid'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='valid'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

## Create the full GAN

In [0]:
generator = make_generator_model_seg(_padding = 'valid')
discriminator = make_discriminator_model_seg()

For now, we use the `binary_crossentropy` discriminator loss as before, which is fine. 

You can copy the Dice loss over from the other notebooks to try and get better performance instead of also using the `cross_entropy` loss for the generator below.

In [0]:
# The discriminator puts out a value from [0,1] for the batch of real and fake segmentations. This is as before.
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
  
# The loss for the generator now takes real and fake segmentations as input to calc their XEnt.
def generator_loss(real_output, fake_output):
    return cross_entropy(real_output, fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

We don't have to modify the checkpointing.

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

Here is a modified image output function.

In [0]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0], cmap='gray') # Remove the scaling/offset
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

# Draw a random sample of images from the test set for testing
example_test_slices = np.random.randint(0,len(x_test),16)
seed = x_test[example_test_slices]
seed = np.lib.pad(seed[...,np.newaxis], [(0,0), (20,20), (20,20), (0,0)], 'reflect')
print(seed.shape)

Formerly, we only wanted to learn images from random number, this time we replace the random input with an image, and the desired output with the segmentation mask.

Recipe:
1. Replace `noise` with CT image (first index in tuple)
1. `generated_images` are the output of the AE
1. Submit the correct images/masks to the loss calculation

The rest should fall into place :-)


In [0]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled", speeding it up about three-fold.
@tf.function
def train_step_seg(images):
    # noise = tf.random.normal([BATCH_SIZE, noise_dim]) # [1]

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      # generated_images = generator(noise, training=True) # [1]
      generated_images = generator(images[0], training=True)

      # real_output = discriminator(images, training=True) [2]
      real_output = discriminator(images[1], training=True)
      fake_output = discriminator(generated_images, training=True)

      # gen_loss = generator_loss(fake_output) [3]
      gen_loss = generator_loss(images[1], generated_images)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train_seg(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step_seg(image_batch)
  
    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

## Train the model

In [0]:
train_seg(train_dataset_liver, EPOCHS)

## Next Steps
You may try to improve on the model. 

1. Plot the loss. Any sign what's wrong?

   Plotting the loss can be done in inline tensorboard, like seen here: https://www.tensorflow.org/tensorboard/get_started
1. Pooling and upsampling are known to hamper performance. 
   
   Replace with strided convolutions, and with Upconvolutions.
1. Next, BatchNorm should be employed in the Generator.
1. The cross entropy loss is not optimal. 

   Replace it with a Dice loss.
